In [1]:
from pathlib import Path

import warm_pixels as wp

# We assume our data is in a directory parallel to warm_pixels called 'hst_acs_datasets'
data_directory = Path.cwd().parent.parent / "hst_acs_datasets"

# We can create an object to access all datasets in that directory
source = wp.Source(
    data_directory
)

# We can get a list of datasets from that source
datasets = list(source)

# Each dataset comprises several images captured on a given date
dataset = datasets[0]

In [2]:
# We can get the images from a dataset
images = dataset.images
image = images[0]

# Each image has 4 quadrants which can be addressed by letter
quadrant_a = image.quadrant("A")

# We can get an array for that quadrant
array = quadrant_a.array()

In [3]:
# We can search for warm pixels by making a WarmPixels object from our dataset
warm_pixels = wp.WarmPixels(
    datasets=datasets,
)

# Groups of quadrants with one item for each dataset.
groups = warm_pixels.all_groups()

# A list of groups for the first dataset. There is one group for each combination
# of quadrants. By default there is only one group with quadrants ABCD
first_dataset_groups = groups[0]
group = first_dataset_groups[0]

2022-10-03 15:11:44,498 - richmassey60491 - INFO - Retrieving groups
2022-10-03 15:11:44,499 - richmassey61092 - INFO - Retrieving groups
2022-10-03 15:11:44,500 - 04_2017 - INFO - Retrieving groups
2022-10-03 15:11:44,501 - 10_2017 - INFO - Retrieving groups
2022-10-03 15:11:44,502 - 07_2020_original - INFO - Retrieving groups


In [4]:
import json

# Warm pixel lines are extracted and grouped into bins by their attributes.
# 1) Find warm pixels
# 2) Extract the CTI trail corresponding to those warm pixels
# 3) Keep only those lines which appear consistently across multiple images
# 4) Stack warm pixels into bins with similar CCD row, flux, background and date
stacked_lines = group.stacked_lines()

# Each stacked line has a dictionary representation
stacked_line_dicts = [
    stacked_line.dict
    for stacked_line
    in stacked_lines
]

# Let's see what the first one looks like
print(json.dumps(stacked_line_dicts[0], indent=4))

# We can save these to a JSON
with open("stacked_lines.json", "w+") as f:
    json.dump(stacked_line_dicts, f)

{
    "location": [
        13.0,
        0
    ],
    "date": 2453963.758414352,
    "background": 31.30540652532858,
    "flux": 992.075,
    "data": [
        46.933043816970624,
        46.3701649771964,
        48.711080079945226,
        48.11518701894584,
        48.14361054867522,
        48.00110239950205,
        46.16310970820016,
        47.16358109918174,
        46.49024043290878,
        46.863087745614244,
        45.796840479854254,
        45.55346051503446,
        1309.4024294348244,
        48.59285309531201,
        46.63910977027298,
        47.10570908449849,
        48.99711399716423,
        48.20789077340186,
        47.29295786328075,
        48.53875401695496,
        48.920490584648604,
        48.141066131134835,
        47.34273778298907,
        50.227607541836136,
        47.744377164578616
    ],
    "noise": [
        0.5256967407012534,
        0.4280892624149115,
        0.789855949552933,
        0.6217203894640145,
        0.5786548351791704,
   

In [5]:
from autocti.dataset_1d.dataset_1d.dataset_1d import Dataset1D

# We can find the size if our CCD from the shape of the array we loaded earlier
size = array.shape[0]

# Using this and a pixel line dictionary we can create a Dataset1D. This can
# then be used with autocti.
dataset_1d = Dataset1D.from_pixel_line_dict(
    stacked_line_dicts[0],
    size=size,
)

# Better yet we can create a Dataset1D for every stacked pixel line we found
dataset_1ds = [
    Dataset1D.from_pixel_line_dict(
        stacked_line_dict,
        size=array.shape[0],
    )
    for stacked_line_dict
    in stacked_line_dicts
]

<class 'int'>
